In [3]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2584/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [1]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_30_9_0,0.999416,0.770696,0.998027,0.998694,0.998432,0.000291,0.136125,0.000914,0.000737,0.000825,0.004904,0.017048,1.000346,0.017126,1182.287000,3155.447621,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
3,model_4_6_6,0.999395,0.811075,0.998223,0.998492,0.998375,0.000301,0.112154,0.000744,0.000705,0.000725,0.007125,0.017344,1.000085,0.017425,3534.217933,9487.544748,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
4,model_4_6_7,0.999395,0.811202,0.998098,0.998388,0.998261,0.000301,0.112078,0.000796,0.000754,0.000775,0.007117,0.017348,1.000085,0.017428,3534.217182,9487.543998,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
5,model_4_6_8,0.999394,0.811152,0.997991,0.998308,0.998169,0.000301,0.112108,0.000841,0.000792,0.000816,0.007106,0.017362,1.000085,0.017442,3534.213994,9487.540810,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
7,model_4_6_9,0.999390,0.811043,0.997889,0.998226,0.998079,0.000304,0.112173,0.000884,0.000829,0.000857,0.007065,0.017422,1.000086,0.017502,3534.200120,9487.526936,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,model_3_5_1,0.998484,0.798796,0.998806,0.998737,0.998774,0.000754,0.119443,0.000613,0.000608,0.000610,0.010346,0.027460,1.000213,0.027587,3532.380076,9485.706891,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
431,model_3_5_0,0.998464,0.798761,0.998822,0.998742,0.998784,0.000764,0.119464,0.000605,0.000605,0.000605,0.010394,0.027640,1.000216,0.027768,3532.353950,9485.680766,"Hidden Size=[40, 39], regularizer=0.0, learnin..."
433,model_21_7_9,0.998453,0.778197,0.998079,0.997469,0.997775,0.000769,0.131672,0.001038,0.001520,0.001279,0.011861,0.027735,1.005009,0.027863,582.340248,1543.536845,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
435,model_33_8_0,0.998437,0.807780,0.998880,0.998512,0.998728,0.000777,0.114110,0.000663,0.000786,0.000724,0.010618,0.027882,1.000549,0.028011,1684.319065,4510.372443,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
